---
## Title: "1 Characteristics of Time Series 1.2 Time Series Statistical Models"
author: "Aaron Smith"
date: '2022-10-22'
output: html_document
---

The objective of time series analysis is to build mathematical models that provide plausible descriptions for sample data.

For example, we may consider a time series as a sequence of random variables, 

$$
x_1, x_2, x_3, \ldots
$$

where 

* the random variable  $x_1$ denotes the value taken by the series at the first time point, 
* the variable $x_2$ denotes the value for the second time period, 
* $x_3$ denotes the value for the third time period, and so on. 

In general, a collection of random variables, 

$$
\{x_t\},
$$
indexed by $t$ is referred to as a stochastic process.

In this text, $t$ will typically be discrete and vary over the integers 

$$
t = 0, ±1, ±2,\ldots,
$$

or some subset of the integers. 

The observed values of a stochastic process are referred to as a realization of the stochastic process.

We display a sample time series graphically by plotting the values of the random variables on the vertical axis, with the time scale on the horizontal axis.

We connect the values at adjacent time periods to reconstruct visually a hypothetical continuous time series that might have produced these values discretely. 

The appearance of data can be changed completely by adopting an insufficient sampling rate. We have all seen wheels in movies appear to be turning
backwards because of the insufficient number of frames sampled by the camera. This phenomenon leads to a distortion called aliasing.

The fundamental visual characteristic distinguishing different series is their differing degrees of smoothness. One possible explanation for this smoothness is that it is being induced by correlation between adjacent points, so the value of the series at time $t$, $x_t$, depends on the past values 

$$
x_{t−1}, x_{t−2},\ldots.
$$

#### Example 1.8 White Noise (3 Flavors)

* white noise: uncorrelated, mean 0, constant variance wrt time
* iid white noise: independent, identically distributed, mean 0, constant variance wrt time
* Gaussian white noise: independent, Gaussian distributed, mean 0, constant variance wrt time

A simple kind of generated series might be a collection of uncorrelated random variables, $w_t$, with mean 0 and finite variance $\sigma_w^2$.

The time series generated from uncorrelated variables is used as a model for noise, called white noise.

We shall denote this process as 

$$
w_t ∼ wn(0, \sigma_w^2).
$$

The designation white originates from the analogy with white light and all possible periodic oscillations are present with equal strength.

We will sometimes require the noise to be independent and identically distributed (iid) random variables with mean 0 and variance $\sigma_w^2$. We distinguish this by writing 

$$
w_t ∼ iid(0, \sigma_w^2)
$$

or by saying white independent noise or iid noise.

Gaussian white noise: wherein the $w_t$ are independent normal random variables, with mean 0 and variance $\sigma_w^2$, 

$$
w_t ∼ iid N(0, \sigma_w^2)
$$

```{r}
v_rnorm = rnorm(
  n = 500,
  mean = 0,
  sd = 1
)                  # 500 N(0,1) variates
astsa::tsplot(
  x = v_rnorm,
  main = "Gaussian white noise"
)
```

#### Example 1.9 Moving Averages and Filtering

We might replace the white noise series $w_t$ by a moving average that smooths the series. For example, consider replacing $w_t$ by an average of its current value and its immediate neighbors in the past and future.

$$
v_t = \dfrac{w_{t-1} + w_t + w_{t-1}}{3}
$$

The averaged series shows a smoother version of the first series, reflecting the fact that the slower oscillations are more apparent and some of the faster oscillations are taken out. We begin to notice a similarity to the SOI in the figure, or perhaps, to some of the fMRI
series in the figure.

A linear combination of values in a time series is called a filtered series.

```{r}
filter_rnorm = filter(
  x = v_rnorm,
  sides = 2,
  filter = rep(
    x = 1/3,
    times = 3
  )
)  # moving average
#par(mfrow=c(2,1))
astsa::tsplot(
  x = v_rnorm,
  col = 4,
  main = "white noise"
)
astsa::tsplot(
  x = filter_rnorm,
  col = 4,
  ylim = c(
    -3,3
  ),
  main = "moving average"
)
```

#### Example 1.10 Autoregressions

If we see a predominate oscillatory-sinusoidal behavior, then moving average is not an appropriate model. We will use a different type of model for quasi-periodic behavior: autoregression

Suppose we consider a white noise series $w_t$ as input and calculate the output using the second-order equation

$$
x_t = x_{t−1} − 0.9 x_{t−2} + w_t,\\
t \in 1,\ldots,500
$$

This Equation represents a regression of the current value $x_t$ of a time series as a function of the past two values of the series, hence the term autoregression.

A problem with start up values exists here because the values depend on $x_0$ and $x_1$

Note the periodic behavior of the series.

The data is obtained by a filter of white noise.

The function filter uses zeros for the initial values. In this case, 

$$
x_1 = w_1 \\
x_2 = x_1 + w_2 = w_1 + w_2
$$ 

In this example, the initial values do not satisfy the autoregression equation.

```{r}
v_rnorm = rnorm(
  n = 550,
  mean = 0,
  sd = 1
)  # 50 extra to avoid startup problems
filter_rnorm = filter(
  x = v_rnorm,
  filter = c(
    1,-0.9
  ),
  method = "recursive"
)[-(1:50)]
astsa::tsplot(
  x = filter_rnorm,
  col = 4,
  main = "autoregression"
)
```

#### Example 1.11 Random Walk with Drift

A model for analyzing trends is the random walk with drift model given by

$$
x_t = \delta + x_{t−1} + w_t \\
t = 1,2,3,\ldots \\
x_0 = 0 \\
w_t \text{ is white noise (random movement)} \\
\delta \text{ is the drift term}
$$

When $\delta = 0$ the model is called a random walk.

The term random walk comes from the fact that, when $\delta = 0$, the value of the time series at time $t$ is the value of the series at time $t − 1$ plus a completely random movement determined by $w_t$.

We may rewrite a random walk with drift as a cumulative sum of white noise variates.

$$
x_t = \delta t + \sum_{j = 1}^{t} w_j
$$

```{r}
set.seed(
  seed = 823
) # so you can reproduce the results
v_rnorm = rnorm(
  n = 200
)
cumsum_rnorm = cumsum(
  x = v_rnorm
) # two commands in one line
wd = v_rnorm + 0.2;
v_randomwalkdrift = cumsum(
  x = wd
)
astsa::tsplot(
  x = v_randomwalkdrift,
  ylim = c(
    -5,55
  ),
  main = "random walk",
  ylab = ''
)
lines(
  x = cumsum_rnorm,
  col = 4
) 
clip(
  x1 = 0,
  x2 = 200,
  y1 = 0,
  y2 = 50
)
abline(
  h = 0,
  col = 4,
  lty = 2
)
abline(
  a = 0,
  b = 0.2,
  lty = 2
)
```

#### Example 1.12 Signal in Noise

Many realistic models for generating time series assume an underlying signal with some consistent periodic variation, contaminated by adding a random noise. 

Consider the model

$$
x_t = 2 cos \left( 2\pi\dfrac{t+15}{50} \right) + w_t \\
t = 1,2,\ldots,500
$$

The cosine term is the signal.

The sinusoidal waveform can be written as

$$
A cos \left( 2\pi \omega t + \phi \right) \\
A \text{ amplitude} \\
\omega \text{ frequency of oscillations, one cycle per } 1/\omega \\
\phi \text{ phase shift}
$$

The degree to which the signal is obscured depends on the amplitude of the signal and the size of $\sigma_w$.

The ratio of the amplitude of the signal to $\sigma_w$ (or some function of the ratio) is sometimes called the signal-to-noise ratio (SNR); the larger the signal-to-noise ratio, the easier it is to detect the signal. 

Typically, we will not observe the signal but the signal obscured by noise.

We use spectral analysis to detect periodic signals

$$
x_t = s_t + v_t \\
s_t \text{ signal} \\
v_t \text{ white noise or autoregressive}
$$

When $v_t$ is autoregressive, the model is a state-space model.

```{r}
cs = 2*cos(
  x = 2*pi*(1:500)/50 + 0.6*pi
)
v_rnorm = rnorm(
  n = 500,
  mean = 0,
  sd = 1
)
#par(mfrow=c(3,1))
astsa::tsplot(
  x = cs,
  ylab = "",
  main = expression(
    x[t]==2*cos(2*pi*t/50 + 0.6*pi)
  )
)
astsa::tsplot(
  x = cs + v_rnorm,
  ylab = "",
  main = expression(
    x[t]==2*cos(2*pi*t/50 + 0.6*pi)+N(0,1)
  )
)
astsa::tsplot(
  x = cs + 5*v_rnorm,
  ylab = "",
  main = expression(
    x[t]==2*cos(2*pi*t/50 + 0.6*pi)+N(0,25)
  )
)
```